In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Load the Fashion MNIST dataset
(x_train, _), (_, _) = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255.0  # Normalize pixel values to be between 0 and 1
x_train = np.expand_dims(x_train, axis=-1)  # Add channel dimension

# Define the generator and discriminator models
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, input_dim=latent_dim, activation='relu'))
    model.add(layers.Dense(784, activation='sigmoid'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

def build_discriminator(img_shape):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Define the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Build and compile the discriminator
img_shape = (28, 28, 1)
discriminator = build_discriminator(img_shape)
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build and compile the generator
latent_dim = 100
generator = build_generator(latent_dim)

# Build and compile the GAN model
discriminator.trainable = False
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Training parameters
epochs = 10000
batch_size = 64

for epoch in range(epochs):
    # Generate random noise as input to the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))

    # Generate fake images using the generator
    generated_images = generator.predict(noise)

    # Select a random batch of real images from the dataset
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_images = x_train[idx]

    # Create labels for the generated and real images
    labels_real = np.ones((batch_size, 1))
    labels_fake = np.zeros((batch_size, 1))

    # Train the discriminator on real and fake images separately
    d_loss_real = discriminator.train_on_batch(real_images, labels_real)
    d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)

    # Calculate the total discriminator loss
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator by fooling the discriminator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    labels_gan = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(noise, labels_gan)

    # Print progress and save generated images at certain intervals
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")
        # Save generated images
        gen_imgs = generator.predict(np.random.normal(0, 1, size=(16, latent_dim)))
        gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale generated images to [0, 1]
        fig, axs = plt.subplots(4, 4)
        count = 0
        for i in range(4):
            for j in range(4):
                axs[i, j].imshow(gen_imgs[count, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                count += 1
        plt.show()

Output hidden; open in https://colab.research.google.com to view.